In [1]:
# Load data from Excel file
def GetData(DataFile, DataWorksheet):
    Width = LoadFromExcel(DataFile, DataWorksheet, 'Width')
    Length = LoadFromExcel(DataFile, DataWorksheet, 'Length')
    Weight = LoadFromExcel(DataFile, DataWorksheet, 'Weight')
    Width.columns = ['Item']
    Length.columns = ['Item']
    Weight.columns = ['Item']
    return Width, Length, Weight

In [2]:
# Define model data, assigning all data to the Model
def DefineModelData(Model, Width, Length, Weight):
    Model.Item = pyo.Set(initialize = range(0, len(Width)))
    Model.Candidate = pyo.Set(initialize = range(0, len(Width) + 1))   # + 1 because we include an extra candidate to ensure feasibility
    Model.Width = pyo.Param(Model.Item, within = pyo.NonNegativeIntegers, mutable = True)
    Model.Length = pyo.Param(Model.Item, within = pyo.NonNegativeIntegers, mutable = True)
    Model.Weight = pyo.Param(Model.Item, within = pyo.NonNegativeReals, mutable = True)
    
    Model.Baseline = 0   # Total weighted area of all items
    for i in Model.Item:
        Model.Width[i] = Width['Item'][i]
        Model.Length[i] = Length['Item'][i]
        Model.Weight[i] = Weight['Item'][i]
        Model.Baseline += Model.Width[i] * Model.Length[i] * Model.Weight[i]

    # Define candidate product sizes using width and length of items
    Model.CandidateWidth = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)
    Model.CandidateLength = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)
    Model.CandidateArea = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)
    for i in Model.Item:
        Model.CandidateWidth[i] = Width['Item'][i]
        Model.CandidateLength[i] = Length['Item'][i]
        Model.CandidateArea[i] = Width['Item'][i] * Length['Item'][i]

    # Extra candidate to ensure feasibility. Dimensions will depend on how the data is sorted
    MaxWidth = 0
    MaxLength = 0
    for i in Model.Item:
        MaxWidth = max(MaxWidth, Width['Item'][i])
        MaxLength = max(MaxLength, Length['Item'][i])
    Model.CandidateWidth[len(Width)] = max(MaxWidth, MaxLength)   # We take the max because we want the sizes sorted so that width >= length
    Model.CandidateLength[len(Width)] = min(MaxWidth, MaxLength)  # Conversely, we take the min because we want length <= width
    Model.CandidateArea[len(Width)] = MaxWidth * MaxLength